In [18]:
# Nur benötigt für GoLab
!pip3 install tira ir-datasets python-terrier nltk


In [19]:
# Imports
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
from nltk.stem import SnowballStemmer

import pyterrier as pt
import pandas as pd


# PyTerrier initialisieren
ensure_pyterrier_is_loaded()
tira = Client()

# PyTerrier starten
if not pt.started():
    pt.init()

# Pandas Display-Einstellungen
pd.set_option('display.max_colwidth', 0)

# Dataset von der TIRA Plattform abrufen
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')


In [20]:
# Benutzerdefinierter SnowballStemmer (Porter2)
class NLTKStemmerWrapper:
    def __init__(self):
        self.stemmer = SnowballStemmer("english")

    def __call__(self, text):
        return " ".join([self.stemmer.stem(word) for word in text.split()])

In [21]:
# Benutzerdefinierten Stemmer instanziieren
custom_stemmer = NLTKStemmerWrapper()

In [ ]:
# Vorverarbeitung: SnowballStemmer auf den Text anwenden
def stem_text(text):
    return custom_stemmer(text)

In [22]:
# Index erstellen mit einer PyTerrier-Pipeline
def create_index(corpus):
    # Indexer erstellen
    indexer = pt.IterDictIndexer("/tmp/index2", overwrite=True)
    
    # Corpus durchlaufen und Stemmer anwenden
    processed_corpus = [
        {'docno': doc['docno'], 'text': stem_text(doc['text'])}
        for doc in corpus
    ]

    # Indexieren
    index_ref = indexer.index(processed_corpus)
    return index_ref


In [23]:
# Index erstellen
index_ref = create_index(pt_dataset.get_corpus_iter())

# Index laden
index = pt.IndexFactory.of(index_ref)

# BM25 Retrieval Modell initialisieren
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

ir-lab-sose-2024/ir-acl-anthology-20240504-training documents:  66%|██████▋   | 84161/126958 [02:10<01:02, 684.46it/s] 

In [ ]:
# Gestemmte Queries vorbereiten
topics = pt_dataset.get_topics()

In [ ]:
# Übertragen der Stemming-Funktion auf die Queries
def apply_stem_to_queries(df):
    df['text'] = df['text'].apply(stem_text)
    df['title'] = df['title'].apply(stem_text)
    df['query'] = df['query'].apply(stem_text)
    return df

In [ ]:
# Gestemmte Queries anwenden
stemmed_topics = apply_stem_to_queries(topics)

In [ ]:
# Experiment durchführen
experiment = pt.Experiment(
    [bm25], 
    stemmed_topics,  # Verwende die gestemmten Topics hier
    pt_dataset.get_qrels(), 
    eval_metrics=['P_1000', 'map', 'recip_rank'],
    names=['BM25'],
    baseline=0
)

In [ ]:
# Ergebnisse des Experiments anzeigen
print("Ergebnisse des Experiments:")
print(experiment)

In [ ]:
# Output runfile für das Deployment auf TIRA
persist_and_normalize_run(experiment, system_name='bm25-custom-stemmer', default_output='../../runs')

In [ ]:
# Direkte Abfrage mit gestemmter Query
query = "retrieval system improving effectiveness"
stemmed_query = stem_text(query)
print(f"Original Query: {query}")
print(f"Stemmed Query: {stemmed_query}")

In [ ]:
# Abfrage auf dem Index mit gestemmter Query
manual_run = bm25.search(stemmed_query)
print("Ergebnisse der manuellen Abfrage:")
print(manual_run.head(10))